Autores: Bruno Leal Fonseca & Guilherme Namen Pimenta

# Data Understanding & Data Preparation

A base de dados consiste em pastas contendo o mês e ano dos registros feitos pelos radares de Belo Horizonte. Cada mês contém subpastas para os respectivos dias, onde é possível encontrar conjuntos de arquivos no formato JSON. Nelas há registros de todos os radares. Um exemplo da estrutura de pastas é dado abaixo: 

ABRIL_2022/ 

┣ 20220401/ 

┃ ┣ 20220401_00.json 

┃ ┣ 20220401_01.json 

┃ ┣ 20220401_02.json 

┃ ┣ 20220401_03.json 

┃ ┣ 20220401_04.json 

┃ ┣ 20220401_05.json  

Nota-se que cada arquivo JSON contem os dados de uma faixa de horário no respectivo dia.

Uma das primeiras suposições é que se deve encontrar associações diretas entre as velocidades registradas com os horários, tipos de veículos e localização na via. Um determinado local pode ser mais comum encontrar velocidades mais altas em determinados horários ou em determinados tipos de veículos. Velocidades mais baixas ou altas também podem ser encontradas em horários bem comuns.Além do mais, pode-se encontrar padrões frequentes em casos que se registra infrações por excesso de velocidade.

Foram adquiridos dados do ano de 2022 inteiro, mesclando todo os JSON em dataframes. Contudo o volume de dados é muito grande, portanto, usar biblioteca como pandas não será o suficiente. A solução foi usar um framework mais adequado para Big Data, o `Polars`. Uma vez que os dado estiverem processados, transformados e as informações necessárias filtradas, eles serão levados até a modelagem.

Os atributos da base são as seguintes:
- ID EQP: Número de identificação do equipamento no sistema de 
processamento. 
- DATA HORA: Data e hora de ocorrência do evento. 
- MILESEGUNDO: Milésimo de segundo de ocorrência do evento. 
- FAIXA: Faixa de circulação da via onde ocorreu o evento. 
- ID DE ENDEREÇO: Localização do equipamento no sistema de processamento. 
- VELOCIDADE DA VIA: Velocidade regulamentada para a via. 
- VELOCIDADE AFERIDA: Velocidade medida pelo equipamento. 
- CLASSIFICACAO: Tipo de veículo registrado pelo equipamento (Automóvel/Motocicleta/Caminhão ou ônibus). 
- TAMANHO: Comprimento do veículo detectado pelo equipamento. 
- NUM SERIE: Número de identificação do equipamento. 
- LATITUDE: Localização de coordenada geográfica latitude do equipamento. 
- LONGITUDE: Localização de coordenada geográfica longitude do equipamento. 
- ENDEREÇO: Localização da via onde o equipamento se encontra instalado. 
- SENTIDO: Sentido da via fiscalizado pelo equipamento. 

As colunas que serão usadas serão as seguintes:
- "ENDEREÇO"
- "SENTIDO"
- "FAIXA"
- "CLASSIFICAÇÃO"
- "TAMANHO"
- "VELOCIDADE DA VIA"
- "VELOCIDADE AFERIDA"

Os dados contínuos serão discretizados de forma arbitrária usando visualização de histogramas. E serão gerados dois novos atributos a partir da DATA HORA, VELOCIDADE DA VIA e VELOCIDADE AFERIDA:
- "PERIODO DIA"
- "ULTRAPASSOU LIMITE"
Espera-se obter informações mais ricas usando dessa nova representação discreta dos horários, e minerar padrões em casos onde houveram infrações de trânsito.

Os códigos abaixo consistem em uma dataclass usada para modelar o banco de dados usando `Polars` dataframes, e a chamada da classe para gerar arquivos CSV que serão usados na modelagem

In [ ]:
import os
from glob import glob
from dataclasses import dataclass
from tqdm import tqdm
import polars as pl


@dataclass
class DataLoader:
    dir: str
    target: str

    def list_files(self):
        months_folders = os.listdir(self.dir)
        json_files_list = []

        for m_folder in months_folders:
            m_folder = os.path.join(self.dir, m_folder)
            if os.path.isdir(m_folder):
                d_folder_list = os.listdir(m_folder)
                for d_folder in d_folder_list:
                    d_folder = os.path.join(m_folder, d_folder)
                    json_files = glob(f"{d_folder}/*.json")
                    json_files_list.extend(json_files)

        return json_files_list

    def filter_location(self, p_df: pl.DataFrame):
        filtered_df = (
            p_df.lazy()
            .filter(pl.col("ENDEREÇO").str.contains(f"{self.target}"))
            .collect()
        )

        return filtered_df

    def check_above_speed_limite(self, p_df: pl.DataFrame):
        df = p_df.with_columns(
            pl.col("VELOCIDADE AFERIDA").cast(pl.Float32),
            pl.col("VELOCIDADE DA VIA").cast(pl.Float32),
        )
        df = df.with_columns(
            (pl.col("VELOCIDADE AFERIDA") > pl.col("VELOCIDADE DA VIA")).alias(
                "ULTRAPASSOU LIMITE"
            )
        )

        return df

    def categorize_time_of_day_extended(self, hour):
        if 0 <= hour < 6:
            return "Madrugada"
        elif 6 <= hour < 9 or 17 <= hour < 20:
            return "Pico"
        elif 9 <= hour < 12:
            return "Manhã"
        elif 12 <= hour < 17:
            return "Tarde"
        else:
            return "Noite"

    def discretize_datetime(self, p_df: pl.DataFrame):
        df = p_df.with_columns(
            pl.col("DATA HORA").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S")
        )

        df = (
            df.lazy()
            .with_columns(
                pl.col("DATA HORA")
                .dt.hour()
                .map_elements(
                    self.categorize_time_of_day_extended, return_dtype=pl.String
                )
                .alias("PERIODO DIA")
            )
            .collect()
        )

        return df

    def load_data(self) -> pl.DataFrame:
        json_files_list = self.list_files()
        dataframes_list = []

        for file in tqdm(json_files_list):
            df = pl.read_json(file)
            filtered_df = self.filter_location(df)
            filtered_df = self.check_above_speed_limite(filtered_df)

            filtered_df = self.discretize_datetime(filtered_df)

            filtered_df = filtered_df[
                "PERIODO DIA",
                "ENDEREÇO",
                "SENTIDO",
                "FAIXA",
                "CLASSIFICAÇÃO",
                "TAMANHO",
                "VELOCIDADE DA VIA",
                "VELOCIDADE AFERIDA",
                "ULTRAPASSOU LIMITE",
            ]
            dataframes_list.append(filtered_df)

        months_df = pl.concat(dataframes_list)

        return months_df

In [ ]:
import os
from dotenv import load_dotenv
from dataLoader import DataLoader


if __name__ == "__main__":
    load_dotenv("bhTrafficDataMining/humanAttempt/dataProcessing/config/.env")
    data_loader = DataLoader("bhTrafficDataMining/data", os.environ["TARGET"])
    df = data_loader.load_data()
    df.write_csv(
        "bhTrafficDataMining/humanAttempt/dataProcessing/dataProcessed/ABRIL_2022.csv",
    )
    print(df["ENDEREÇO"].value_counts())


# Modeling

## Minerando Padrões em Casos de Infração de Trânsito

Usando dos dados salvos em CSV, pode-se filtrar apenas os casos onde houveram infrações de trânsito. O Python conta com a biblioteca `mlxtend` que permite minerar padrões frequentes com algoritmos vistos em aula e gerar regras de associação. É possível selecionar qual métrica será usada para gerar as regras de associação entre os itemsets frequentes, no caso foi usada a confiança com um limiar arbitrário de 75%. A saída retornada pela biblioteca é um dataframe que contêm as regras de associação, ordenadas pelo suporte, contendo informações de várias métricas comumente usadas para avaliar regras de associação, dentre elas estão:

- support(A->C) = support(A+C) [aka 'support'], range: [0, 1]
- confidence(A->C) = support(A+C) / support(A), range: [0, 1]
- lift(A->C) = confidence(A->C) / support(C), range: [0, inf]
- leverage(A->C) = support(A->C) - support(A)*support(C), range: [-1, 1]
- conviction = [1 - support(C)] / [1 - confidence(A->C)], range: [0, inf]
- zhangs_metric(A->C) = leverage(A->C) / max(support(A->C)(1-support(A)), support(A)(support(C)-support(A->C))) range: [-1,1]

Abaixo está o código da dataclass usada para preprocessar os dados, transformando os atributos citados anteriormente em one-hot-encoders para cada classe dos atributos. Em seguida está a chamada dessa classe, levando-os para a execução dos algoritmos.

In [ ]:
from glob import glob
from dataclasses import dataclass
from tqdm import tqdm
import polars as pl


@dataclass
class DataPreprocessor:
    dir: str

    def load_database(self, p_percentage) -> pl.DataFrame:
        csv_files_list = glob(f"{self.dir}/*.csv")
        percent_index = int(round(len(csv_files_list) * p_percentage))
        dataframes_list = []
        for file in tqdm(csv_files_list[:percent_index]):
            df = pl.read_csv(file)
            df = self.discretize_speed(df)
            dataframes_list.append(df)

        return pl.concat(dataframes_list)

    def get_speep_intervals(self, p_speed):
        if 0 <= p_speed < 50:
            return "velocidadeModerada"
        elif 50 <= p_speed < 100:
            return "velocidadeAlta"
        elif p_speed >= 100:
            return "velocidadeAltissima"

    def discretize_speed(self, p_df: pl.DataFrame):
        df = (
            p_df.lazy()
            .with_columns(
                pl.col("VELOCIDADE AFERIDA")
                .map_elements(self.get_speep_intervals, return_dtype=pl.String)
                .alias("VELCOIDADE")
            )
            .collect()
        )

        return df

    def drop_undesired_columns(
        self, p_df: pl.DataFrame, p_undesired_columns: list
    ) -> pl.DataFrame:
        return p_df.drop(p_undesired_columns)

    def get_preprocessed_database(self, p_percentage: float, p_undesired_columns: list):
        df = self.load_database(p_percentage)
        df = self.drop_undesired_columns(df, p_undesired_columns)
        df = df.to_dummies()
        df = df.cast(pl.Boolean)

        return df


if __name__ == "__main__":
    data_preprocessor = DataPreprocessor(
        "bhTrafficDataMining/humanAttempt/dataProcessing/dataProcessed"
    )

    df = data_preprocessor.get_preprocessed_database(
        0.5, p_undesired_columns=["VELOCIDADE AFERIDA", "VELOCIDADE DA VIA", "TAMANHO"]
    )
    print(df)


In [ ]:
import polars as pl
from mlxtend.frequent_patterns import fpgrowth, association_rules
from preprocessor import DataPreprocessor


if __name__ == "__main__":
    data_preprocessor = DataPreprocessor(
        "bhTrafficDataMining/humanAttempt/dataProcessing/dataProcessed"
    )

    df = data_preprocessor.get_preprocessed_database(
        1.0, p_undesired_columns=["VELOCIDADE AFERIDA", "VELOCIDADE DA VIA", "TAMANHO"]
    )
    df = df.lazy().filter(pl.col("ULTRAPASSOU LIMITE_true") == True).collect()
    df = df.drop(["ULTRAPASSOU LIMITE_true", "ULTRAPASSOU LIMITE_false"])
    df = df.to_pandas()
    print(df)

    frequent_itemsets = fpgrowth(df, min_support=0.25, use_colnames=True, verbose=1)
    print(frequent_itemsets)
    frequent_itemsets.to_csv("bhTrafficDataMining/humanAttempt/resultados.csv")

    rules = association_rules(
        frequent_itemsets,
        metric="confidence",
        min_threshold=0.75,
        num_itemsets=df.shape[0],
    )
    print(rules)
    rules.to_csv("bhTrafficDataMining/humanAttempt/association_rules_conf75.csv")
